In [44]:
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, GRU, Conv1D, Embedding, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rand
import numpy as np
import pronouncing
import markovify
import textstat
import math

import re
import syllables

In [28]:
print(re.sub(r'[^a-zA-Z]', '', lyric_path))

UserspatricknaylorDesktopMetalDatalyricstxt


In [29]:
lyric_path = '/Users/patricknaylor/Desktop/Metal/Data/lyrics_1.txt'

with open(lyric_path, 'r') as file:
    song = (file.read())
    lyrics = song.replace('\ufeff', '').split("\n")


for line in lyrics:
    line = re.sub(r'[^a-zA-Z]', '', line)

#print(lyrics)

In [30]:
markov_model = markovify.NewlineText(str("\n".join(lyrics)), well_formed=False, state_size=3)
sentence = markov_model.make_sentence(tries=100)
print(sentence)

For we've fallen on the wrath of god to he who discovers the key


In [31]:
sequences = lyrics
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(sequences)

V = len(tokenizer.word_index)+1
seq = pad_sequences(tokenizer.texts_to_sequences(sequences), maxlen=30)

In [32]:
train_X, train_y = seq[:, :-1], tf.keras.utils.to_categorical(seq[:, -1], num_classes=V)

print(train_X.shape, train_y.shape)

(57446, 29) (57446, 20761)


In [33]:
D = 512

#Simple RNN
T = train_X.shape[1]
i = Input(shape=(T,))
x = Embedding(V, D)(i)
x = Dropout(0.2)(x)
x = SimpleRNN(150)(x)
x = Dense(V, activation="softmax")(x)
rnn_model = Model(i, x)

adam = tf.keras.optimizers.Adam(learning_rate=0.001)

rnn_model.compile(optimizer=adam, metrics=["accuracy"], loss="categorical_crossentropy")
rnn_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 29)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 29, 512)           10629632  
                                                                 
 dropout_1 (Dropout)         (None, 29, 512)           0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 150)               99450     
                                                                 
 dense_1 (Dense)             (None, 20761)             3134911   
                                                                 
Total params: 13863993 (52.89 MB)
Trainable params: 13863993 (52.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.optimizers import Adam 


import warnings
warnings.filterwarnings('ignore')
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

es = EarlyStopping(monitor="loss", mode="min", verbose=1, patience=20)

In [35]:
rnn_r = rnn_model.fit(train_X, train_y, epochs=100,callbacks=[learning_rate_reduction,es])

Epoch 1/100
1796/1796 [==============================] - 65s 36ms/step - loss: 7.7908 - accuracy: 0.0375 - lr: 0.0010
Epoch 2/100
1796/1796 [==============================] - 63s 35ms/step - loss: 6.3063 - accuracy: 0.0995 - lr: 0.0010
Epoch 3/100
1796/1796 [==============================] - 65s 36ms/step - loss: 5.1941 - accuracy: 0.1880 - lr: 0.0010
Epoch 4/100
1796/1796 [==============================] - 67s 37ms/step - loss: 4.2384 - accuracy: 0.2911 - lr: 0.0010
Epoch 5/100
1796/1796 [==============================] - 66s 37ms/step - loss: 3.4573 - accuracy: 0.4022 - lr: 0.0010
Epoch 6/100
1796/1796 [==============================] - 70s 39ms/step - loss: 2.8371 - accuracy: 0.4996 - lr: 0.0010
Epoch 7/100
1796/1796 [==============================] - 67s 37ms/step - loss: 2.3712 - accuracy: 0.5712 - lr: 0.0010
Epoch 8/100
1796/1796 [==============================] - 275s 153ms/step - loss: 2.0250 - accuracy: 0.6278 - lr: 0.0010
Epoch 9/100
1796/1796 [==============================]

In [36]:
def calc_readability(input_bars):
  avg_readability = 0
  for bar in input_bars:
    avg_readability += textstat.automated_readability_index(bar)
  return avg_readability / len(input_bars)

In [37]:
def calc_rhyme_density(bars):
  total_syllables = 0
  rhymed_syllables = 0
  for bar in bars:
    for word in bar.split():
      p = pronouncing.phones_for_word(word)
      if len(p) == 0:
        break
      syllables = pronouncing.syllable_count(p[0])
      total_syllables += syllables
      has_rhyme = False
      for rhyme in pronouncing.rhymes(word):
        if has_rhyme:
          break
        for idx, r_bar in enumerate(bars):
          if idx > 4:
            break
          if rhyme in r_bar:
            rhymed_syllables += syllables
            has_rhyme = True
            break
  return rhymed_syllables/total_syllables

In [50]:
def generate_bar(seed_phrase, model, length_of_bar):
  syl = 0
  while syl <= length_of_bar:
    seed_tokens = pad_sequences(tokenizer.texts_to_sequences([seed_phrase]), maxlen=29)
    output_p = model.predict(seed_tokens)
    output_word = np.argmax(output_p, axis=1)[0]-1
    syl += syllables.estimate(str(list(tokenizer.word_index.items())[output_word][0]))
    seed_phrase += " " + str(list(tokenizer.word_index.items())[output_word][0])
  return seed_phrase

In [39]:
def score_bar(input_bar, artists_bars, artists_avg_readability, artists_avg_rhyme_idx):
  gen_readability = textstat.automated_readability_index(input_bar)
  gen_rhyme_idx = calc_rhyme_density(input_bar)
  comp_bars = compare_bars(input_bar, artists_bars)

  # Scores based off readability, rhyme index, and originality. The lower the score the better.
  bar_score = (artists_avg_readability - gen_readability) + (artists_avg_rhyme_idx - gen_rhyme_idx) + comp_bars
  return bar_score

In [40]:
def compare_bars(input_bar, artists_bars):
  '''
    input_bars are the fire bars our AI generates
    artists_bars are the original bars for the artist

    The lower the score the better! We want unique bars
  '''
  # Converts sentences to matrix of token counts
  avg_dist = 0
  total_counted = 0
  for bar in artists_bars:
    v = CountVectorizer()
    # Vectorize the sentences
    word_vector = v.fit_transform([input_bar, bar])
    # Compute the cosine distance between the sentence vectors
    cos_dist = 1-pdist(word_vector.toarray(), 'cosine')[0]
    if not math.isnan(cos_dist):
      avg_dist += 1-pdist(word_vector.toarray(), 'cosine')[0]
      total_counted += 1
  return avg_dist/total_counted

In [48]:
def generate_song( model, intro_bar, artists_bars, length_of_bar, length_of_song=20, min_score_threshold=-0.2, max_score_threshold=0.2, tries=5):
  artists_avg_readability = calc_readability(artists_bars)
  artists_avg_rhyme_idx = calc_rhyme_density(artists_bars)
  fire_song = [intro_bar]
  bar_lengths = [7, 5]
  cur_tries = 0
  candidate_bars = []

  while len(fire_song) < 3:
    try:
        seed_sentence = markov_model.make_sentence(tries=100).split(" ")
        print('Seed Sentence: ', seed_sentence)
        seed_sentence = " ".join(fire_song[-1].split(' ')[-3:]) + " ".join(seed_sentence[:2])
    except:
        pass
    cur_tries += 1
    bar = generate_bar(seed_sentence, model, rand.randrange(4, bar_lengths[len(fire_song)-1]))
    #print(bar)
    bar_score = score_bar(bar, lyrics, artists_avg_readability, artists_avg_rhyme_idx) 
    candidate_bars.append((bar_score, bar))


    if bar_score <= max_score_threshold and bar_score >= min_score_threshold and syllables.estimate(' '.join(bar.split(' ')[3:]) == bar_lengths[len(fire_song) - 1]):
      fire_song.append(' '.join(bar.split(' ')[3:]))
      cur_tries = 0
      print("Generated Bar: ", len(fire_song))

    if cur_tries >= tries:
      lowest_score = np.Infinity
      best_bar = ""
      for bar in candidate_bars:
        if bar[0] < lowest_score & syllables.estimate(' '.join(bar[1].split(' ')[3:])) == bar_lengths[len(fire_song) - 1]:
          best_bar = bar[1]
          candidate_bars = []
      
      fire_song.append(' '.join(best_bar.split(' ')[3:]))
      print("Generated Bar: ", len(fire_song))
      cur_tries = 0
      
  print("Generated song with avg rhyme density: ", calc_rhyme_density(fire_song), "and avg readability of: ", calc_readability(fire_song))
  return fire_song

In [49]:
print('test')
rnn = generate_song( rnn_model, 'Death is consuming the earth', lyrics, length_of_bar =12 , tries=25)

print("Song Generated with SimpleRNN:")
for line in rnn:
  print(line)


test
Seed Sentence:  ['Forged', 'into', 'steel,', 'with', 'the', 'edge', 'of', 'the', 'abyss', 'open', 'and', 'swallow', 'our', 'enemies', 'whole']
1/1 [==============================] - 0s 121ms/step


AttributeError: 'numpy.int64' object has no attribute 'lower'

In [43]:
### TODO
#Clean code
#comment
#reformat